In [1]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)

In [2]:
import pandas as pd
import re
import numpy as np


In [ ]:
path = '.result'
file_list = os.listdir(path)

In [5]:
tmp = pd.DataFrame()
df_eval = pd.DataFrame()
acc_list = []
def sc_calc_acc_condition_with_temp_with_sc(llm_model, few_shot_n, test_n, q_src_yn, ver, p_ver, sc_num, temp, excel_ver):
    path = './result'
    # ./result/sc_l_result_4_15_Y_30_sys_prompt8_0.01_ver1_0.csv
    file_list = os.listdir(path)
    opt_file = [x for x in file_list if x.startswith(f'sc_{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn}_{ver}_{p_ver}_{sc_num}_{temp}_{excel_ver}')]
    opt_file = [x for x in opt_file if x.endswith(f'.csv')]

    df = pd.DataFrame()


    # pattern = re.compile(r'.*_[0-9]\.csv$')
    # opt_file = [x for x in opt_file if pattern.match(x)]
    
    
    if len(opt_file)>0 : 
        for f in opt_file:
            print(f)
            tmp = pd.read_csv(f'{path}/{f}', index_col =0)
            tmp = tmp.dropna()

            tmp['gold'] = tmp['answer_encode'].apply(lambda x : re.sub(r'[^012]', '', x))
            tmp['o_result'] = tmp['result'].apply(lambda x : re.sub(r'[^012]', '', x))
            tmp = tmp[tmp['o_result'].isin(['1', '0', '2'])]

            
            gold_df = tmp[['id', 'gold']].drop_duplicates()
            chk_cnt = tmp.groupby(['id', 'o_result']).count().reset_index()[['id', 'o_result', 'question']]
            chk_cnt = chk_cnt.rename(columns = {'question': 'cnt'})
            chk_cnt = chk_cnt[chk_cnt['cnt'] == sc_num]
            chk_cnt = chk_cnt.sort_values(by = ['id', 'cnt'], ascending=[True, False]).groupby(['id']).head(1)
            df_eval = pd.merge(gold_df, chk_cnt, on = ['id'])

            df_eval['equal_yn'] = np.where(df_eval['gold']==df_eval['o_result'], 1, 0)
            acc = (df_eval['equal_yn'].sum()/df_eval.shape[0])*100  
            acc_list.append(acc)
            df = pd.concat([df, df_eval], axis =0)
            
        df['equal_yn'] = np.where(df['gold']==df['o_result'], 1, 0)
        acc = (df['equal_yn'].sum()/df.shape[0])*100            
        print(f'{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn} : ', acc)


In [6]:
# (target=task, args=('l', 4, 30, 'Y', 100, 'sys_prompt9', 5, 0.01, 'ver3'))
sc_calc_acc_condition_with_temp_with_sc('v', 4, 10, 'Y', 10, 'sys_prompt8', 5,  0.01, 'ver5')

In [ ]:
print(acc_list)

In [ ]:
# sc_calc_acc_condition_with_temp_with_sc('l', 4, 15, 'Y', 30, 'sys_prompt8', 5,  0.01, 'ver1')
# # process3 = Process(target=task, args=('l', 4, 15, 'Y', 30, 'sys_prompt8', 3, 0.01, 'ver1'))
sc_calc_acc_condition_with_temp_with_sc('l', 4, 30, 'Y', 100, 'sys_prompt8', 5,  0.01, 'ver2')
# 그래프 그려서 넣어두기... 분포....

In [ ]:
# (target=task, args=('l', 4, 30, 'Y', 100, 'sys_prompt9', 5, 0.01, 'ver3'))
sc_calc_acc_condition_with_temp_with_sc('l', 4, 10, 'Y', 10, 'sys_prompt10', 5,  0.01, 'ver4')

In [ ]:
# (target=task, args=('l', 4, 30, 'Y', 100, 'sys_prompt9', 5, 0.01, 'ver3'))
sc_calc_acc_condition_with_temp_with_sc('l', 4, 10, 'Y', 10, 'sys_prompt10', 5,  0.01, 'ver1')

In [ ]:
# (target=task, args=('l', 4, 30, 'Y', 100, 'sys_prompt9', 5, 0.01, 'ver3'))
sc_calc_acc_condition_with_temp_with_sc('l', 4, 10, 'Y', 10, 'sys_prompt10', 5,  0.01, 'ver5')

In [ ]:
sc_calc_acc_condition_with_temp_with_sc('v', 4, 10, 'Y', 10, 'sys_prompt8', 5,  0.01, 'ver5')
# 그래프 그려서 넣어두기... 분포....

In [ ]:
# (target=task, args=('l', 4, 30, 'Y', 100, 'sys_prompt9', 5, 0.01, 'ver3'))
sc_calc_acc_condition_with_temp_with_sc('l', 4, 10, 'Y', 10, 'sys_prompt10', 5,  0.01, 'ver1')

In [ ]:
np.mean(acc_list)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

ax.hist(acc_list
    ,  color = 'lightgray'
    # ,  edgecolor = 'black'
      ) # hist type

ax.set_title("Histogram of Acc Test", # add title
             fontsize = 12.5,
             weight = 'bold',
             pad = 10,
             loc = 'center')
ax.set_xlabel("Accuracy", # add x-axis label
              fontsize = 12.5)
ax.set_ylabel("# of test", # add y-axis label
              fontsize = 12.5)

ax.axvline(np.mean(acc_list), # add vertical line
           color = 'blue',
           linestyle = '--',
           linewidth = 1.5)
fig.text(0.35, 0.82, color = 'blue', fontsize = 10,  s='Mean: ' + str(round(np.mean(acc_list), 2))) #


plt.show();


In [ ]:
# def calc_acc_condition(llm_model, few_shot_n, test_n, q_src_yn, p_ver):
#     opt_file = [x for x in file_list if x.startswith(f'{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn}')]
#     opt_file = [x for x in opt_file if x.endswith(f'_{p_ver}.csv')]

#     if len(opt_file)>0 : 
#         df = pd.DataFrame()

#         for f in opt_file:
#             tmp = pd.read_csv(f'./result/{f}', index_col =0)
#             df = pd.concat([df, tmp], axis =0)
        
#         df['gold'] = df['answer_encode'].apply(lambda x : re.sub(r'[^0-9]', '', x))
#         df['o_result'] = df['result'].apply(lambda x : re.sub(r'[^0-9]', '', x))
#         df['equal_yn'] = np.where(df['gold']==df['o_result'], 1, 0)
#         acc = (df['equal_yn'].sum()/df.shape[0])*100
#         print(f'{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn} : ', acc)


In [ ]:
def calc_acc_condition(llm_model, few_shot_n, test_n, q_src_yn, p_ver):
    opt_file = [x for x in file_list if x.startswith(f'{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn}')]
    opt_file = [x for x in opt_file if x.endswith(f'_{p_ver}.csv')]

    if len(opt_file)>0 : 
        df = pd.DataFrame()

        for f in opt_file:
            tmp = pd.read_csv(f'./result/{f}', index_col =0)
            tmp['gold'] = tmp['answer_encode'].apply(lambda x : re.sub(r'[^0-9]', '', x))
            tmp['o_result'] = tmp['result'].apply(lambda x : re.sub(r'[^0-9]', '', x))
            tmp[['gold', 'o_result']] = tmp[['gold', 'o_result']].astype(int)
            df_eval = tmp.groupby('id').mean('o_result').reset_index()
            df_eval['o_result'] = df_eval['o_result'].round(0)
            df = pd.concat([df, tmp], axis =0)
            
        df['equal_yn'] = np.where(df['gold']==df['o_result'], 1, 0)
        acc = (df['equal_yn'].sum()/df.shape[0])*100            
        print(f'{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn} : ', acc)


In [ ]:
calc_acc_condition('c' , 2, 10, 'N', 'sys_prompt1')
calc_acc_condition('c' , 2, 10, 'Y', 'sys_prompt1')
calc_acc_condition('c' , 3, 10, 'N', 'sys_prompt1')
calc_acc_condition('c' , 3, 10, 'Y', 'sys_prompt1')


calc_acc_condition('l' , 1, 10, 'Y', 'sys_prompt1')
calc_acc_condition('l' , 2, 10, 'Y', 'sys_prompt1')
calc_acc_condition('l' , 3, 10, 'Y', 'sys_prompt1')
calc_acc_condition('l' , 4, 10, 'Y', 'sys_prompt1')

calc_acc_condition('l' , 1, 10, 'N', 'sys_prompt1')
calc_acc_condition('l' , 2, 10, 'N', 'sys_prompt1')
calc_acc_condition('l' , 3, 10, 'N', 'sys_prompt1')
calc_acc_condition('l' , 4, 10, 'N', 'sys_prompt1')


In [ ]:
calc_acc_condition('c' , 2, 10, 'N', 'sys_prompt2')
calc_acc_condition('c' , 2, 10, 'Y', 'sys_prompt2')
calc_acc_condition('c' , 3, 10, 'N', 'sys_prompt2')
calc_acc_condition('c' , 3, 10, 'Y', 'sys_prompt2')

calc_acc_condition('l' , 1, 10, 'Y', 'sys_prompt2')
calc_acc_condition('l' , 2, 10, 'Y', 'sys_prompt2')
calc_acc_condition('l' , 3, 10, 'Y', 'sys_prompt2')
calc_acc_condition('l' , 4, 10, 'Y', 'sys_prompt2')


calc_acc_condition('l' , 1, 10, 'N', 'sys_prompt2')
calc_acc_condition('l' , 2, 10, 'N', 'sys_prompt2')
calc_acc_condition('l' , 3, 10, 'N', 'sys_prompt2')
calc_acc_condition('l' , 4, 10, 'N', 'sys_prompt2')


In [ ]:
calc_acc_condition('c' , 2, 10, 'N', 'sys_prompt3')
calc_acc_condition('c' , 2, 10, 'Y', 'sys_prompt3')
calc_acc_condition('c' , 3, 10, 'N', 'sys_prompt3')
calc_acc_condition('c' , 3, 10, 'Y', 'sys_prompt3')

calc_acc_condition('l' , 1, 10, 'Y', 'sys_prompt3')
calc_acc_condition('l' , 2, 10, 'Y', 'sys_prompt3')
calc_acc_condition('l' , 3, 10, 'Y', 'sys_prompt3')
calc_acc_condition('l' , 4, 10, 'Y', 'sys_prompt3')


calc_acc_condition('l' , 1, 10, 'N', 'sys_prompt3')
calc_acc_condition('l' , 2, 10, 'N', 'sys_prompt3')
calc_acc_condition('l' , 3, 10, 'N', 'sys_prompt3')
calc_acc_condition('l' , 4, 10, 'N', 'sys_prompt3')


In [ ]:
calc_acc_condition('c' , 2, 10, 'N', 'sys_prompt4')
calc_acc_condition('c' , 2, 10, 'Y', 'sys_prompt4')
calc_acc_condition('c' , 3, 10, 'N', 'sys_prompt4')
calc_acc_condition('c' , 3, 10, 'Y', 'sys_prompt4')

calc_acc_condition('l' , 1, 10, 'Y', 'sys_prompt4')
calc_acc_condition('l' , 2, 10, 'Y', 'sys_prompt4')
calc_acc_condition('l' , 3, 10, 'Y', 'sys_prompt4')
calc_acc_condition('l' , 4, 10, 'Y', 'sys_prompt4')


calc_acc_condition('l' , 1, 10, 'N', 'sys_prompt4')
calc_acc_condition('l' , 2, 10, 'N', 'sys_prompt4')
calc_acc_condition('l' , 3, 10, 'N', 'sys_prompt4')
calc_acc_condition('l' , 4, 10, 'N', 'sys_prompt4')


In [ ]:
def sc_calc_acc_condition(llm_model, few_shot_n, test_n, q_src_yn, p_ver):
    opt_file = [x for x in file_list if x.startswith(f'sc_{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn}')]
    opt_file = [x for x in opt_file if x.endswith(f'_{p_ver}.csv')]
    df = pd.DataFrame()
    if len(opt_file)>0 : 
        print(opt_file)
        for f in opt_file:
            tmp = pd.read_csv(f'./result/{f}', index_col =0)
            tmp['gold'] = tmp['answer_encode'].apply(lambda x : re.sub(r'[^0-9]', '', x))
            tmp['o_result'] = tmp['result'].apply(lambda x : re.sub(r'[^0-9]', '', x))
            tmp[['gold', 'o_result']] = tmp[['gold', 'o_result']].astype(int)
            df_eval = tmp.groupby('id').mean('o_result').reset_index()
            df_eval['o_result'] = df_eval['o_result'].round(0)
            df_eval['equal_yn'] = np.where(df_eval['gold']==df_eval['o_result'], 1, 0)
            acc = (df_eval['equal_yn'].sum()/df_eval.shape[0])*100  
            # print(acc)
            df = pd.concat([df, df_eval], axis =0)
            
        df['equal_yn'] = np.where(df['gold']==df['o_result'], 1, 0)
        acc = (df['equal_yn'].sum()/df.shape[0])*100            
        print(f'{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn} : ', acc)


In [ ]:
sc_calc_acc_condition('l' , 1, 10, 'Y', 'sys_prompt4')
sc_calc_acc_condition('l' , 2, 10, 'Y', 'sys_prompt4') # temperature 0.2
sc_calc_acc_condition('l' , 3, 10, 'Y', 'sys_prompt4')

sc_calc_acc_condition('l' , 1, 10, 'N', 'sys_prompt4')
sc_calc_acc_condition('l' , 2, 10, 'N', 'sys_prompt4')
sc_calc_acc_condition('l' , 3, 10, 'N', 'sys_prompt4')

In [ ]:
sc_calc_acc_condition('l' , 1, 5, 'Y', 'sys_prompt4')
sc_calc_acc_condition('l' , 2, 5, 'Y', 'sys_prompt4')
sc_calc_acc_condition('l' , 3, 5, 'Y', 'sys_prompt4')

# sc_l_result_1_10_Y_0_sys_prompt4.csv
# (f'./result/sc_{llm_model}_result_{few_shot_n}_{self.test_n}_{q_src_yn}_{self.version}_{self.p_ver}.csv')

sc_calc_acc_condition('l' , 1, 5, 'N', 'sys_prompt4')
sc_calc_acc_condition('l' , 2, 5, 'N', 'sys_prompt4')
sc_calc_acc_condition('l' , 3, 5, 'N', 'sys_prompt4')


In [ ]:

sc_calc_acc_condition('l' , 2, 5, 'Y', 'sys_prompt5')
sc_calc_acc_condition('l' , 2, 5, 'N', 'sys_prompt5')


In [ ]:

sc_calc_acc_condition('c' , 2, 10, 'Y', 'sys_prompt4')
sc_calc_acc_condition('c' , 2, 10, 'N', 'sys_prompt4')


In [ ]:
sc_calc_acc_condition('c' , 2, 10, 'Y', 'sys_prompt7')
sc_calc_acc_condition('c' , 2, 10, 'N', 'sys_prompt7')


In [ ]:

sc_calc_acc_condition('l' , 2, 10, 'Y', 'sys_prompt7')
sc_calc_acc_condition('l' , 2, 10, 'N', 'sys_prompt7')


In [ ]:

sc_calc_acc_condition('l' , 4, 10, 'Y', 'sys_prompt7')
sc_calc_acc_condition('l' , 4, 10, 'N', 'sys_prompt7')


In [ ]:
def sc_calc_acc_condition_with_temp(llm_model, few_shot_n, test_n, q_src_yn, p_ver, temp):
    opt_file = [x for x in file_list if x.startswith(f'sc_{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn}')]
    opt_file = [x for x in opt_file if x.endswith(f'_{p_ver}_{temp}.csv')]
    df = pd.DataFrame()
    if len(opt_file)>0 : 
        for f in opt_file:
            print(f)
            tmp = pd.read_csv(f'./result/{f}', index_col =0)
            tmp['gold'] = tmp['answer_encode'].apply(lambda x : re.sub(r'[^0-9]', '', x))
            tmp['o_result'] = tmp['result'].apply(lambda x : re.sub(r'[^0-9]', '', x))
            tmp[['gold', 'o_result']] = tmp[['gold', 'o_result']].astype(int)
            df_eval = tmp.groupby('id').mean('o_result').reset_index()
            df_eval['o_result'] = df_eval['o_result'].round(0)
            df_eval['equal_yn'] = np.where(df_eval['gold']==df_eval['o_result'], 1, 0)
            acc = (df_eval['equal_yn'].sum()/df_eval.shape[0])*100  
            print(acc)
            df = pd.concat([df, df_eval], axis =0)
            
        df['equal_yn'] = np.where(df['gold']==df['o_result'], 1, 0)
        acc = (df['equal_yn'].sum()/df.shape[0])*100            
        print(f'{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn} : ', acc)


In [ ]:
sc_calc_acc_condition_with_temp('l', 2, 10, 'Y','sys_prompt4', 0)

In [ ]:
llm_model, few_shot_n,test_n, q_src_yn, p_ver, temp = 'l',2, 10, 'Y','sys_prompt4', 0
opt_file = [x for x in file_list if x.startswith(f'sc_{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn}')]
opt_file = [x for x in opt_file if x.endswith(f'_{p_ver}_{temp}.csv')]
df = pd.DataFrame()
if len(opt_file)>0 : 
    for f in opt_file:
        print(f)
        tmp = pd.read_csv(f'./result/{f}', index_col =0)
        tmp['gold'] = tmp['answer_encode'].apply(lambda x : re.sub(r'[^0-9]', '', x))
        tmp['o_result'] = tmp['result'].apply(lambda x : re.sub(r'[^0-9]', '', x))
        tmp[['gold', 'o_result']] = tmp[['gold', 'o_result']].astype(int)
        tmp['equal_yn'] = np.where(tmp['gold']==tmp['o_result'], 1, 0)
        acc = (tmp['equal_yn'].sum()/tmp.shape[0])*100  
        df = pd.concat([df, tmp], axis =0)

df.to_csv('youcandoit.csv')

In [ ]:
sc_calc_acc_condition_with_temp('l', 2, 10, 'Y','sys_prompt4', 0.5)